In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print(f'length of text: {len(text)} characters')

length of text: 1115394 characters


In [4]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
vocab = sorted(set(text))

print(f"unique characters: {len(vocab)}")

unique characters: 65


In [6]:
# create mapping from char to index and index to char
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

# Process the Text
text_as_int = np.array([char2idx[c] for c in text])

In [7]:
for char in char2idx:
  print(f"{repr(char)}: {char2idx[char]}")

'\n': 0
' ': 1
'!': 2
'$': 3
'&': 4
"'": 5
',': 6
'-': 7
'.': 8
'3': 9
':': 10
';': 11
'?': 12
'A': 13
'B': 14
'C': 15
'D': 16
'E': 17
'F': 18
'G': 19
'H': 20
'I': 21
'J': 22
'K': 23
'L': 24
'M': 25
'N': 26
'O': 27
'P': 28
'Q': 29
'R': 30
'S': 31
'T': 32
'U': 33
'V': 34
'W': 35
'X': 36
'Y': 37
'Z': 38
'a': 39
'b': 40
'c': 41
'd': 42
'e': 43
'f': 44
'g': 45
'h': 46
'i': 47
'j': 48
'k': 49
'l': 50
'm': 51
'n': 52
'o': 53
'p': 54
'q': 55
'r': 56
's': 57
't': 58
'u': 59
'v': 60
'w': 61
'x': 62
'y': 63
'z': 64


In [8]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [9]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text) // (seq_length+1)

# create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

it = iter(sequences)
seq = next(it)
seq.numpy().shape

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


(101,)

In [11]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input, target in dataset.take(1):
  print(f"input text: {repr(''.join(idx2char[input.numpy()]))}")
  print(f"target text: {repr(''.join(idx2char[target.numpy()]))}")

input text: 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
target text: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [13]:
# Create Training Batches
# Batch Size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [14]:
# length of the vocabulary in chars
vocab_size = len(vocab)
print(f"vocab size: {vocab_size}")

# the embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

vocab size: 65


In [15]:
# Define a model using tf.keras.Sequential
"""
  tf.keras.layers.Embedding:  The input layer. A trainable lookup table that will
                              map the numbers of each character to a vector with
                              'embedding_dim' dimensions
  tf.keras.layers.GRU:        A type of RNN with size units=rnn_units (You can also use
                              use a LSTM layer here)
  tf.keras.layers.Dense:      The output layer, with vocab_size outputs
"""

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])

  return model

In [16]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE
    )

In [17]:
for input_batch, target_batch in dataset.take(1):
  example_batch_predictions = model(input_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size")

(64, 100, 65) # (batch_size, sequence_length, vocab_size


In [18]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([12, 10, 42, 29, 26, 64, 11, 36, 32, 57, 16,  4, 62,  2, 34, 51, 28,
        7, 58, 19, 38, 22, 13, 63, 29, 23, 33,  5, 19, 64, 22, 29, 48, 44,
       60, 24, 50, 46,  5, 26, 43, 32, 15, 38, 54, 31, 55, 30, 36, 10, 25,
       27, 32, 48, 52, 59, 58, 27, 13, 37, 42,  0, 61, 41, 58,  0, 11,  8,
       51, 41, 48, 42, 49,  5, 33, 20, 48, 11, 62, 36, 11, 37, 21, 34, 30,
        9,  1, 61, 13, 19, 55, 41, 19, 45, 51, 31, 22,  4, 19, 20])

In [19]:
print("Input: \n", repr("".join(idx2char[input_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices])))

Input: 
 'Tis very true: thou didst it excellent.\nWell, you are come to me in a happy time;\nThe rather for I h'

Next Char Predictions: 
 "?:dQNz;XTsD&x!VmP-tGZJAyQKU'GzJQjfvLlh'NeTCZpSqRX:MOTjnutOAYd\nwct\n;.mcjdk'UHj;xX;YIVR3 wAGqcGgmSJ&GH"


In [20]:
# Train the model
# Attach an optimizer and loss function
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_batch, example_batch_predictions)
print(f"Prediction shape: {example_batch_predictions.shape}, # (batch_size, sequence_length, vocab_size)")
print(f"scalar_loss: {example_batch_loss.numpy().mean()}")

Prediction shape: (64, 100, 65), # (batch_size, sequence_length, vocab_size)
scalar_loss: 4.175633430480957


In [21]:
model.compile(optimizer='adam', loss=loss)

In [22]:
# Configure Checkpoints

# Directory where the checkpoints will be saved
checkpoint_dir = "./training_checkpoints"

# name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [23]:
# Execute the Training
EPOCHS = 10

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 9s 52ms/step - loss: 2.6754
Epoch 2/10
172/172 [==============================] - 9s 52ms/step - loss: 1.9627
Epoch 3/10
172/172 [==============================] - 9s 53ms/step - loss: 1.6975
Epoch 4/10
172/172 [==============================] - 9s 53ms/step - loss: 1.5489
Epoch 5/10
172/172 [==============================] - 9s 54ms/step - loss: 1.4599
Epoch 6/10
172/172 [==============================] - 9s 54ms/step - loss: 1.4001
Epoch 7/10
172/172 [==============================] - 9s 54ms/step - loss: 1.3541
Epoch 8/10
172/172 [==============================] - 9s 55ms/step - loss: 1.3171
Epoch 9/10
172/172 [==============================] - 9s 55ms/step - loss: 1.2816
Epoch 10/10
172/172 [==============================] - 10s 55ms/step - loss: 1.2500


In [24]:
# Restore the Latest Checkpoint
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [27]:
# function to test the model to generate text
def generate_text(model, start_string):
  # Number of characters to generate
  num_generate = 1000

  #converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text
  # Higher temperatures results in more surprising text
  # Ezperiment to find the best setting
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)

    # remove batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions /= temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])
  
  return (start_string + ''.join(text_generated))

In [28]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: I have do with stain he ware?

LEONTES:
Heas substals, thy grandscreecious;
You are to thy ceast: I hear no match,
And this is ich Priditable sound!

HORTENSIO:
Yet the hear this.

SALTIS:
Stricts their woe;
I must all Henry's nature to thy legs Shame to see it.

DUKE OF AUMERLE:
Our pupposes to be done since,
Can ch it either here in the voluption.

FRIAR LAURENCE:
Fet, I am your blood and that forgiveness, and these demory's vengeance on his tribune.

DUKE VINCENTIO:
Frith, list; all else hath set a house of York!
I partly a left pright! O, it ever
The bleached age the deep of grieving shumper'd,
Where he round faste than affairs.

KING HENRY VI:
What say' King Richard: therefore leave, not thy cate's black with speed
Un present as I say: mother, to reprove this, say, O loval!

GLOUCESTER:
Sir, his wry, 'tis a pilcont revenge,
Who should suich am I think so long as blow it, now by my cols
And prisoner; fear not and credit it live, let when at hand:
My younger dartners in thee 